# Single EV Snapshot

In this notebook, we are adding one EV (modelled as a fixed load) to the feeder and observe voltage changes.

This is a snapshot study, meaning one operating point only.

Outputs:
- Baseline topology plot (colored by electrical distance)
- Topology plot showing EV location
- Voltage profile along feeder (baseline vs EV)
- Simple summary metrics (minimum voltage and voltage violations)


In [3]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

using PowerModelsDistribution
using DataFrames
using Ipopt
using JuMP
using LinearAlgebra
using Plots
using Graphs
using CairoMakie
using Statistics

const PMD = PowerModelsDistribution

  Activating project at `/mnt/c/Users/auc009/OneDrive - CSIRO/Documents/power-models-distribution/pmd_ev_experiments`


PowerModelsDistribution

In [ ]:
# Paths
file = "../data/Three-wire-Kron-reduced/network_1/Feeder_1/Master.dss"
fig_dir = "../results/figures"
mkpath(fig_dir)

"../results/figures"

## Parse feeder and apply voltage bounds

Let's parse the OpenDSS feeder and apply voltage bounds on phases and neutral.

We will keep the feeder four wire, so we do not Kron reduce the neutral.


In [5]:
eng4w = PMD.parse_file(file, transformations=[PMD.transform_loops!])

println("Buses: ", length(get(eng4w, "bus", Dict())))
println("Lines: ", length(get(eng4w, "line", Dict())))
println("Loads: ", length(get(eng4w, "load", Dict())))

# Keep consistent base scaling (same idea as your baseline)
eng4w["settings"]["sbase_default"] = 1.0 * 1e3 / eng4w["settings"]["power_scale_factor"]

# Add voltage bounds in per unit
PMD.add_bus_absolute_vbounds!(
    eng4w,
    phase_lb_pu = 0.9,
    phase_ub_pu = 1.1,
    neutral_ub_pu = 0.1
)

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 23 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 24 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 25 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 31 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line48 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : ba

Buses: 65
Lines: 64
Loads: 23


Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "enwl_network_20_feeder_5_4wire"
  "line"           => Dict{String, Any}("line48"=>Dict{String, Any}("length"=>8…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>1.0, "vbases_default"=…
  "files"          => ["../data/networks/lv_small_feeder/Master.dss", "../data/…
  "load"           => Dict{String, Any}("load2"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("32"=>Dict{String, Any}("rg"=>Float64[]…
  "linecode"       => Dict{String, Any}("lc8"=>Dict{String, Any}("b_fr"=>[0.0 0…
  "data_model"     => ENGINEERING

## Helper functions

Let's build:
- a topology graph from lines
- electrical distance from the source bus
- a fixed layout for plotting topology
- voltage extraction from OPF results

In [6]:
function build_lines_df(eng)
    line_dict = get(eng, "line", Dict())
    rows = NamedTuple[]
    for (id, ln) in line_dict
        bus1 = get(ln, "bus1", get(ln, "f_bus", nothing))
        bus2 = get(ln, "bus2", get(ln, "t_bus", nothing))
        if bus1 === nothing || bus2 === nothing
            continue
        end
        b1 = split(String(bus1), ".")[1]
        b2 = split(String(bus2), ".")[1]
        L = float(get(ln, "length", 1.0))
        push!(rows, (Bus1=b1, Bus2=b2, Length=L))
    end
    return DataFrame(rows)
end

function build_graph_and_weights(lines_df::DataFrame)
    buses = unique(vcat(lines_df.Bus1, lines_df.Bus2))
    sort!(buses)
    b2i = Dict(b => i for (i,b) in enumerate(buses))
    i2b = Dict(i => b for (b,i) in b2i)

    g = Graph(length(buses))
    w = Dict{Tuple{Int,Int}, Float64}()

    for r in eachrow(lines_df)
        i = b2i[r.Bus1]
        j = b2i[r.Bus2]
        add_edge!(g, i, j)
        w[(i,j)] = r.Length
        w[(j,i)] = r.Length
    end
    return g, w, b2i, i2b
end

function dijkstra_distances(g::Graph, w::Dict{Tuple{Int,Int},Float64}, s::Int)
    n = nv(g)
    dist = fill(Inf, n)
    prev = fill(0, n)
    visited = falses(n)

    dist[s] = 0.0
    pq = [(0.0, s)]

    while !isempty(pq)
        sort!(pq, by=x->x[1])
        (d,u) = popfirst!(pq)
        if visited[u]
            continue
        end
        visited[u] = true

        for v in neighbors(g, u)
            nd = d + get(w, (u,v), 1.0)
            if nd < dist[v]
                dist[v] = nd
                prev[v] = u
                push!(pq, (nd, v))
            end
        end
    end
    return dist, prev
end

function tree_children(prev::Vector{Int})
    n = length(prev)
    children = [Int[] for _ in 1:n]
    for v in 1:n
        p = prev[v]
        if p != 0
            push!(children[p], v)
        end
    end
    return children
end

function tree_layout(prev::Vector{Int}, dist_km::Vector{Float64}, root::Int)
    children = tree_children(prev)
    y = fill(0.0, length(prev))
    next_y = Ref(0.0)

    function dfs(u::Int)
        if isempty(children[u])
            y[u] = next_y[]
            next_y[] += 1.0
        else
            for c in sort(children[u])
                dfs(c)
            end
            y[u] = mean(y[children[u]])
        end
    end

    dfs(root)
    x = copy(dist_km)
    return x, y
end

function plot_topology_with_markers(
    g::Graph, xpos::Vector{Float64}, ypos::Vector{Float64}, dist_km::Vector{Float64};
    source_idx::Int,
    ev_indices::Vector{Int}=Int[],
    title_str::String="Topology (colored by distance)",
    save_path::Union{Nothing,String}=nothing
)
    fig = Figure(size=(1000, 700))
    ax = Axis(fig[1,1], title=title_str)

    for e in edges(g)
        u = src(e); v = dst(e)
        lines!(ax, [xpos[u], xpos[v]], [ypos[u], ypos[v]], color=:gray50, linewidth=2)
    end

    scatter!(ax, xpos, ypos, color=dist_km, markersize=10)
    scatter!(ax, [xpos[source_idx]], [ypos[source_idx]], markersize=22, color=:black)
    text!(ax, "source", position=(xpos[source_idx], ypos[source_idx] + 0.6), align=(:center, :bottom))

    for (k, ev) in enumerate(ev_indices)
        scatter!(ax, [xpos[ev]], [ypos[ev]], markersize=22, color=:red)
        text!(ax, "EV$(k)", position=(xpos[ev], ypos[ev] + 0.6), align=(:center, :bottom))
    end

    hidedecorations!(ax)
    hidespines!(ax)

    if save_path !== nothing
        save(save_path, fig)
    end
    return fig
end

function busid_to_name_map(math)
    m = Dict{String,String}()
    for (id, b) in math["bus"]
        m[string(id)] = split(get(b, "name", string(id)), ".")[1]
    end
    return m
end

function extract_vm_by_busid(result)
    vm = Dict{String, Vector{Float64}}()
    for (id, b) in result["solution"]["bus"]
        vr = b["vr"]; vi = b["vi"]
        vm[string(id)] = sqrt.(vr.^2 .+ vi.^2)
    end
    return vm
end

function build_buses_dict_for_voltage_plot(id2name::Dict{String,String}, dist_km_byname::Dict{String,Float64}, vm_byid::Dict{String,Vector{Float64}})
    buses_dict = Dict{String, Dict{String,Any}}()
    for (id, vm) in vm_byid
        name = id2name[id]
        vma = length(vm) >= 1 ? vm[1] : NaN
        vmb = length(vm) >= 2 ? vm[2] : NaN
        vmc = length(vm) >= 3 ? vm[3] : NaN
        buses_dict[name] = Dict(
            "distance" => get(dist_km_byname, name, NaN),
            "vma" => [float(vma)],
            "vmb" => [float(vmb)],
            "vmc" => [float(vmc)]
        )
    end
    return buses_dict
end

function plot_voltage_along_feeder_snap(buses_dict, lines_df; t=1, vmin=0.9, vmax=1.1)
    p = plot(legend=false)
    xlabel!("Electrical distance from source (km)")
    ylabel!("Voltage magnitude (p.u.)")
    title!("Voltage drop along feeder (snapshot)")

    colors = [:blue, :red, :black]

    for r in eachrow(lines_df)
        b1 = r.Bus1
        b2 = r.Bus2
        for phase in 1:3
            d1 = buses_dict[b1]["distance"]
            d2 = buses_dict[b2]["distance"]
            v1 = phase==1 ? buses_dict[b1]["vma"][t] : phase==2 ? buses_dict[b1]["vmb"][t] : buses_dict[b1]["vmc"][t]
            v2 = phase==1 ? buses_dict[b2]["vma"][t] : phase==2 ? buses_dict[b2]["vmb"][t] : buses_dict[b2]["vmc"][t]
            if isfinite(d1) && isfinite(d2) && isfinite(v1) && isfinite(v2)
                plot!([d1, d2], [v1, v2], color=colors[phase], marker=:circle, markersize=1)
            end
        end
    end

    maxdist = maximum([b["distance"] for (k,b) in buses_dict if isfinite(b["distance"])])
    plot!([0, maxdist], [vmin, vmin], color=:red, linestyle=:dash)
    plot!([0, maxdist], [vmax, vmax], color=:red, linestyle=:dash)
    return p
end

function add_ev_load!(math, bus_id::String; phase::Int=1, P_kw::Float64=7.0, Q_kvar::Float64=0.0)
    sbase = get(math["settings"], "sbase", 1.0)
    P_pu = P_kw / (sbase * 1000)
    Q_pu = Q_kvar / (sbase * 1000)

    existing = parse.(Int, collect(keys(math["load"])))
    new_id = string(isempty(existing) ? 1 : maximum(existing) + 1)

    math["load"][new_id] = Dict(
        "bus" => bus_id,
        "connections" => [phase],
        "pd" => [P_pu],
        "qd" => [Q_pu],
        "status" => 1,
        "model" => "constant_power"
    )
    return new_id
end

function find_math_bus_id_by_name(id2name::Dict{String,String}, target_name::String)
    for (id, nm) in id2name
        if nm == target_name
            return id
        end
    end
    error("No math bus id found for name '$target_name'")
end

find_math_bus_id_by_name (generic function with 1 method)

## Build topology objects and baseline OPF


In [ ]:
lines_df = build_lines_df(eng4w)
g, w, b2i, i2b = build_graph_and_weights(lines_df)

source_name = "sourcebus"
@assert haskey(b2i, source_name) "sourcebus not found in topology bus names"

source_idx = b2i[source_name]
dist_raw, prev = dijkstra_distances(g, w, source_idx)

# If your line lengths are meters, this converts to km
scale_to_km = 1.0 / 1000
dist_km = dist_raw .* scale_to_km

xpos, ypos = tree_layout(prev, dist_km, source_idx)

dist_km_byname = Dict(i2b[i] => dist_km[i] for i in 1:length(dist_km) if isfinite(dist_km[i]))

# Save baseline topology
plot_topology_with_markers(
    g, xpos, ypos, dist_km;
    source_idx=source_idx,
    ev_indices=Int[],
    title_str="LV Topology (colored by electrical distance)",
    save_path=joinpath(fig_dir, "02_topology_baseline.png")
)

In [ ]:
math_base = PMD.transform_data_model(eng4w; multinetwork=false, kron_reduce=false, phase_project=false)
id2name = busid_to_name_map(math_base)

println("Running baseline OPF...")
res_base = PMD.solve_mc_opf(math_base, IVRENPowerModel, Ipopt.Optimizer)
println("Baseline status: ", res_base["termination_status"])

vm_base = extract_vm_by_busid(res_base)
buses_dict_base = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vm_base)

p_base = plot_voltage_along_feeder_snap(buses_dict_base, lines_df; vmin=0.9, vmax=1.1)
savefig(p_base, joinpath(fig_dir, "02_voltage_profile_baseline.png"))

Max electrical distance (km): 0.043825537


## Add one EV at one location

For this notebook, I start simple.
I place one EV at a chosen bus and compare voltage to the baseline.


In [ ]:
# Pick one bus name for this notebook.
# Options: set it manually after printing, or choose far end by distance.

finite_mask = isfinite.(dist_km)
far_idx = argmax(dist_km .* finite_mask)
ev_bus_name = i2b[far_idx]     # far end bus name
ev_bus_id = find_math_bus_id_by_name(id2name, ev_bus_name)

println("EV bus chosen: ", ev_bus_name, "  (math id ", ev_bus_id, ")")

In [ ]:
math_ev = deepcopy(math_base)
add_ev_load!(math_ev, ev_bus_id; phase=1, P_kw=7.0, Q_kvar=0.0)

println("Running OPF with one EV...")
res_ev = PMD.solve_mc_opf(math_ev, IVRENPowerModel, Ipopt.Optimizer)
println("EV status: ", res_ev["termination_status"])

vm_ev = extract_vm_by_busid(res_ev)
buses_dict_ev = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vm_ev)

In [ ]:
ev_idx = b2i[ev_bus_name]
plot_topology_with_markers(
    g, xpos, ypos, dist_km;
    source_idx=source_idx,
    ev_indices=[ev_idx],
    title_str="Topology with EV location",
    save_path=joinpath(fig_dir, "02_topology_with_ev.png")
)

In [ ]:
function min_phase_voltage(vm_byid::Dict{String,Vector{Float64}})
    return minimum([minimum(vm[1:min(3, length(vm))]) for (id, vm) in vm_byid])
end

function count_voltage_violations(vm_byid::Dict{String,Vector{Float64}}; vmin=0.9, vmax=1.1)
    viol = 0
    for (id, vm) in vm_byid
        bad = false
        for ph in 1:min(3, length(vm))
            if vm[ph] < vmin || vm[ph] > vmax
                bad = true
                break
            end
        end
        if bad
            viol += 1
        end
    end
    return viol
end

println("Baseline min V: ", min_phase_voltage(vm_base))
println("EV min V:       ", min_phase_voltage(vm_ev))
println("EV violations:  ", count_voltage_violations(vm_ev))

## Notes

This notebook confirms the full pipeline works:
- parse feeder
- solve baseline OPF
- add one EV load
- solve again
- compare voltage and show EV location on the topology plot

Next notebook: EV sensitivity by changing the EV location.
